<a href="https://colab.research.google.com/github/swapnilmn/Assignment_1_CS6910/blob/master/Question_2ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 2. Implement a feedforward neural network which takes images from the fashion-mnist data as input and outputs a probability distribution over the 10 classes.

Your code should be flexible such that it is easy to change the number of hidden layers and the number of neurons in each hidden layer.

# Libraries 

In [1]:
from keras.datasets import fashion_mnist, mnist
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split 
import pandas as pd

In [2]:
class OneHotEncoder_from_scratch:
    
    def __init__(self):
        self.categories = None
    def fit(self, X):
        self.categories =[]
        for i in range(X.shape[1]):
            feature_categories =list(set(X[:, i]))
            self.categories.append(feature_categories)
            
    def transform(self, X):
        one_hot_vector = []

        for i in range(X.shape[0]):
            one_hot_row = []
            for j in range(X.shape[1]):

                category_index = self.categories[j].index(X[i, j])
                category_one_hot =[0] *len(self.categories[j])
                category_one_hot[category_index] = 1

                one_hot_row.extend(category_one_hot)
            one_hot_vector.append(one_hot_row)
        return np.array(one_hot_vector)

X = np.array([[1],[2]]) 
enc = OneHotEncoder_from_scratch()
enc.fit(X)
enc.transform(X)

(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()
enc.fit(train_labels.reshape(-1, 1))
enc.transform(train_labels.reshape(-1, 1))
#train_labels.reshape(-1, 1)

4422102/4422102 [==============================] - 0s 0us/step


array([[0, 0, 0, ..., 0, 0, 1],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [10]:
dataset = 'fashion_mnist'

def dataset_type(dataset):
  if dataset == 'fashion_mnist':
      (train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()
  elif dataset == 'mnist':
      (train_images, train_labels), (test_images, test_labels) = mnist.load_data()
  else:
      raise ValueError('Invalid dataset name')
  X_train, X_val, Y_train, Y_val = train_test_split(train_images, train_labels, test_size=0.1, random_state=42)
  train_input = []
  for i in range(len(X_train)):
      train_input.append(list(np.concatenate(X_train[i]).flat))

  val_input = []
  for i in range(len(X_val)):
      val_input.append(list(np.concatenate(X_val[i]).flat))

  test_input = []
  for i in range(len(test_images)):
      test_input.append(list(np.concatenate(test_images[i]).flat))
  Y_train = np.array(Y_train)
  Y_val = np.array(Y_val)
  Y_test = np.array(test_labels)

  X_train = np.array(train_input) / 255.0
  X_test = np.array(test_input) / 255.0
  X_val = np.array(val_input) / 255.0

  enc = OneHotEncoder_from_scratch()
  enc.fit(Y_train.reshape(-1, 1))
  Y_train = enc.transform(Y_train.reshape(-1, 1))
  Y_val = enc.transform(Y_val.reshape(-1, 1))
  Y_test = enc.transform(Y_test.reshape(-1, 1))

  return X_train, X_val, X_test, Y_train, Y_val, Y_test

dataset = 'fashion_mnist'
X_train, X_val, X_test, Y_train, Y_val, Y_test = dataset_type(dataset)

print(Y_train.shape, Y_val.shape, Y_test.shape)
print(X_train.shape, X_val.shape, X_test.shape)

(54000, 10) (6000, 10) (10000, 10)
(54000, 784) (6000, 784) (10000, 784)


In [11]:
# we will set a configuartion dictionary for future use for wandb seerp
# we will take input from that configuration dict

config = {'hidden_layers_size' : [32, 64, 128], # We can modify numbers of neurons each layers, parallaly it will modify number of layers
          'pre_activation_function': 'sigmoid',
          'weight_initialization_method': 'xavier',
          }
# Now we will dynamically add no of hidden layers key 
config['hidden_layers_no'] = len(config['hidden_layers_size'])


In [12]:
class FFNN:
  def __init__(self, X, Y,
               epochs = 100, 
               hidden_layer_count = 4,
               hidden_layers =  [32, 64, 128, 256],
               learning_rate = 0.001,
               batch_size = 32,
               activation='tanh',
               weight_init='random',
               loss = 'cross_entropy',
               weight_decay = 0):
    
    self.inputs =X.shape[1] # Number of inputs
    self.outputs= Y.shape[1] # Number of outputs
    self.epochs = epochs
    self.hidden_layers = hidden_layer_count  # Number of hidden layers 
    self.network_size= [self.inputs] + hidden_layers +[self.outputs] # input layer + hidden layers + output layers
    self.learning_rate = learning_rate
    self.batch_size = batch_size
    self.weights={} # It will create dictionary for weights and biases
    self.weights_h = []
    self.num_classes = Y.shape[1]
    self.weight_init = weight_init
    self.activation_function = activation
    self.loss_function = loss
    self.lambd = weight_decay
    np.random.seed(0)  # We will set seed value so that it will generate same random numebers every time

    self.grad_derivatice={}
    self.update_weights={}
    self.prev_update_weights={}
    for i in range(1,self.hidden_layers+1):
      vw_key, vb_key, mb_key, mw_key = [f'{key}{i}' for key in ['vw', 'vb', 'mb', 'mw']]
      self.update_weights[vw_key]=0
      self.update_weights[vb_key]=0
      self.update_weights[mb_key]=0
      self.update_weights[mw_key]=0
      self.prev_update_weights[vw_key]=0
      self.prev_update_weights[vb_key]=0

    # for creating initial weights
    if self.weight_init == 'random':
      for i in range(1, self.hidden_layers + 2):
          weight_shape = (self.network_size[i - 1], self.network_size[i])
          weight_scale = 0.1
          self.weights[f'W{i}'] = np.random.normal(scale=weight_scale, size=weight_shape)*0.1
          # we are mulliplying ny 0.1 to get less wrights
          bias_shape = (1, self.network_size[i])
          self.weights[f'B{i}'] = np.zeros(bias_shape)

    if self.weight_init == 'Xavier':
      for i in range(1, self.hidden_layers + 2):
          weight_shape = (self.network_size[i - 1], self.network_size[i])
          weight_scale = 0.1
          self.weights[f'W{i}'] = np.random.normal(scale=weight_scale, size=weight_shape)*np.sqrt(1/self.network_size[i-1])
          
          bias_shape = (1, self.network_size[i])
          self.weights[f'B{i}'] = np.zeros(bias_shape)



  def forward_activation(self, X):
      activation_functions = {
          'sigmoid': lambda x: 1.0 / (1.0 + np.exp(-x)),
          'tanh': np.tanh,
          'ReLU': lambda x: np.maximum(0, x)
      }
      activation_function = activation_functions.get(self.activation_function)
      if activation_function:
          return activation_function(X)
      else:
          raise ValueError(f"Unknown activation function '{self.activation_function}'")


  def grad_activation(self, X):
      activation_gradients = {
          'sigmoid': lambda x: x * (1 - x),
          'tanh': lambda x: 1 - np.square(x),
          'ReLU': lambda x: 1.0 * (x > 0)
      }
      gradient_function = activation_gradients.get(self.activation_function)
      if gradient_function:
          return gradient_function(X)
      else:
          raise ValueError(f"Unknown activation function '{self.activation_function}'")

  def softmax(self, X):
    exps =np.exp(X - np.max(X, axis=1, keepdims=True))
    return  exps /np.sum(exps, axis=1, keepdims=True)
  

  def forward_pass(self, X, weights=None):
    #X: shape (batch_size, input_dim)
    if weights is None:

        weights =self.weights
    #weights:a dictionary containing weight and bias parameters for each layer
    self.z = {}
    self.z = {}
    self.z[0] = X
    for i in range(self.hidden_layers):
        self.z[i+1] = self.z[i]@weights[f'W{i+1}'] + weights[f'B{i+1}'] # weights[f'W{i+1}'](input_dim, output_dim) + # weights[f'B{i+1}'](1, output_dim)
        #z is dictionary containing the output of each layer's activationfunction
        self.z[i+1] = self.forward_activation(self.z[i+1]) # self.z[i+1](batch_size, output_dim)
    self.z[self.hidden_layers+1] = self.z[self.hidden_layers] @ weights[f'W{self.hidden_layers+1}'] + weights[f'B{self.hidden_layers+1}']

    #self.z[self.hidden_layers+1](batch_size, output_dim)
    self.z[self.hidden_layers+1]=  self.softmax(self.z[self.hidden_layers+1])

    return self.z[self.hidden_layers+1]

  
model = FFNN(X_train, Y_train,
                  epochs = 1, 
                  hidden_layer_count = 4,
                  hidden_layers =  [32, 64, 128, 256],
                  learning_rate = 0.001,
                  batch_size = 32,
                  activation='ReLU',
                  weight_init='random',
                  loss = 'cross_entropy')
model.forward_pass(X_train)[0]

array([0.10000004, 0.10000008, 0.09999996, 0.09999994, 0.09999997,
       0.10000002, 0.09999998, 0.10000001, 0.10000006, 0.09999994])